In [1]:
import pandas as pd
import re
import numpy as np 
import string
import nltk
import warnings 
from nltk.stem.porter import *
from sklearn.svm import SVC
import pickle 
from sklearn.metrics import accuracy_score 
from sklearn import preprocessing 
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
 
stopword_list = nltk.corpus.stopwords.words('english')


# Pre_processing

In [2]:

def pre_processing(text):
 
    expr = re.compile('\d{2}/\d{2}/\d{4}')
    line = re.sub(expr,'',text )
    expr = re.compile('\d{2}:\d{2}:\d{2}')
    text = re.sub(expr, '', line)
    text = re.sub("\d", "", text)
    tokens = nltk.word_tokenize(text)
    pattern = re.compile('[!"\\#\\$%\\&\'\\(\\)\\*\\+,\\-\\.:;<=>\\?@\\[\\\\\\]\\^_`\\{\\|\\}\\~]')
    tokens = [w for w in tokens if w not in stopword_list]
    filtered_tokens = filter(None, [pattern.sub('', token) for token in tokens])
    filtered_text = ' '.join(filtered_tokens)
 
    return filtered_text

# Inference_code

In [3]:
def integer_convert(predictions):
    
    int_predict = []
    
    for i in range(0,len(predictions)):
        
        result = np.where(predictions[i] == np.amax(predictions[i]))

        int_predict.append(result[0][0])
        
    return int_predict

def inference(input_query):
     #Load file weight
     
        
    filename = 'weights/lbl_enc.sav'
    lbl_enc= pickle.load(open(filename, 'rb'))
    
    filename = 'weights/tfv_weights.sav'
    tfv = pickle.load(open(filename, 'rb'))
    
    filename = 'weights/svd_weights.sav'
    svd = pickle.load(open(filename, 'rb'))
    
    filename = 'weights/scl_weights.sav'
    scl = pickle.load(open(filename, 'rb'))
    
    filename = 'weights/clf_weights.sav'
    clf = pickle.load(open(filename, 'rb'))
    
    input_query = tfv.transform(input_query)
    input_query = svd.transform(input_query)
    input_query = scl.transform(input_query)
    
   
     
    predictions = clf.predict_proba(input_query)
     
    
    int_predict = integer_convert(predictions)
     
    labels =  lbl_enc.inverse_transform(int_predict)
    
    print('Label:{}'.format(labels))
    

# Testing

In [5]:
input_query = '''*** NOTES 22/01/2019 12:16:44 wsluser Action Type: PM Team
*** HELPWWW        Web Access    wsluser
Please configure ORKNEY_NHS_SIP  onto the DLSS
This customer solution is expected to have 2 devices under management.
The network will be connected to  
      
NetcoolRainbow                              


This network uses VoIP 
(delete as appropriate)

These devices and their notifications need to be visible in the following GSAM views, and buckets.
GSAM XXX
BUCKET YYY
e.g. Thurso GSAM , off shore bucket
Please include this statement in all MSFBT contracts.  Auto ticketing must be switched on for this Contract

Thank You



 
*** RESEARCH LOG 28/01/2019 08:03:35 nxmcr08 Action Type: Action Taken
*** UKEEMSESMARTS        Mohamed Riswan  +91 9524304152  nxmcr08
Hi Team,

Please add below network.

ORKNEY_NHS_SIP

in rdl07163app124 - APM15-S-NET-roc

and cbl07163app132 - APM15-S-NET-car

After adding the network, please dispatch the case back to us.

Thanks and Regards,
Mohamed Riswan | SMARTS ASG

 
*** RESEARCH LOG 29/01/2019 08:48:07 nuvjk53 Action Type: Action Taken
*** Master UKGSEMSEMSTR        Vishal Kumar  70440 860 65  nuvjk53
HI team ,

could you please let us know that this network will be managed by ?

management ip or customer ip or both ?

 
*** RESEARCH LOG 29/01/2019 08:56:52 nxmcr08 Action Type: Action Taken
*** UKEEMSESMARTS        Mohamed Riswan  +91 952'''

input_query = pre_processing(input_query)
inference([input_query])

Label:['EMSE SMARTS']
